# xgboost的安装
首先将xgboost的源代码从github上下载到本地：
```
git clone --recursive https://github.com/dmlc/xgboost
```
别忘了加上`--recursive`参数，以下载依赖的`dmlc-core`等模块。
接着，编译xgboost的源代码  
```
cd xgboost  
make -j4 #开4个线程编译
```
编译完成后，将在`xgboost/lib`目录下生成文件`libxgboost.so`。  
接下来我们需要安装xgboost的python接口，具体步骤如下：  
1.进入`xgboost/python-packages`，执行
```
python setup.py install
```
安装过程顺便会将`xgboost/lib/libxgboost.so`拷贝到`/home/little-prince/anaconda2/lib/python2.7/site-packages/xgboost-0.6-py2.7.egg/xgboost`目录下。  
2.可以运行以下代码验证是否安装成功

In [1]:
import xgboost as xgb
print xgb.__version__

0.6


## 安装过程中遇到的错误及解决
1.`/home/little-prince/anaconda2/lib/python2.7/site-packages/scipy/sparse/../../../../libstdc++.so.6: version 'GLIBCXX_3.4.20' not found`  
原因：  
GLIBCXX的版本太老。执行
```
sudo  find / -name  libstdc++.so.6*
```
返回结果：  
```
/usr/share/gdb/auto-load/usr/lib/x86_64-linux-gnu/libstdc++.so.6.0.21-gdb.py
/usr/lib/vmware-tools/lib32/libstdc++.so.6
/usr/lib/vmware-tools/lib32/libstdc++.so.6/libstdc++.so.6
/usr/lib/vmware-tools/lib64/libstdc++.so.6
/usr/lib/vmware-tools/lib64/libstdc++.so.6/libstdc++.so.6
/usr/lib/x86_64-linux-gnu/libstdc++.so.6
/usr/lib/x86_64-linux-gnu/libstdc++.so.6.0.21
find: ‘/run/user/1000/gvfs’: Permission denied
/home/little-prince/anaconda2/lib/libstdc++.so.6
/home/little-prince/anaconda2/lib/libstdc++.so.6.0.19
/home/little-prince/anaconda2/pkgs/libgcc-4.8.5-2/lib/libstdc++.so.6
/home/little-prince/anaconda2/pkgs/libgcc-4.8.5-2/lib/libstdc++.so.6.0.19
/home/little-prince/文档/vmware-tools-distrib/lib/lib32/libstdc++.so.6
/home/little-prince/文档/vmware-tools-distrib/lib/lib32/libstdc++.so.6/libstdc++.so.6
/home/little-prince/文档/vmware-tools-distrib/lib/lib64/libstdc++.so.6
/home/little-prince/文档/vmware-tools-distrib/lib/lib64/libstdc++.so.6/libstdc++.so.6
```
其中，`/usr/lib/x86_64-linux-gnu/libstdc++.so.6.0.21`就是我们需要的。接下来执行如下步骤：  
1）`rm /home/little-prince/anaconda2/lib/libstdc++.so.6`  
2)`cp /usr/lib/x86_64-linux-gnu/libstdc++.so.6.0.21 /home/little-prince/anaconda2/lib/libstdc++.so.6`  

2.`/home/little-prince/anaconda2/bin/../lib/libgomp.so.1: version `GOMP_4.0' not found`  
原因：  
同上，版本太老。  
执行
```
sudo find / -name libgomp.so.1*
```
找到  
```
/usr/lib/x86_64-linux-gnu/libgomp.so.1.0.0
/usr/lib/x86_64-linux-gnu/libgomp.so.1
find: ‘/run/user/1000/gvfs’: Permission denied
/home/little-prince/anaconda2/lib/libgomp.so.1.0.0
/home/little-prince/anaconda2/lib/libgomp.so.1
/home/little-prince/anaconda2/pkgs/libgcc-4.8.5-2/lib/libgomp.so.1.0.0
/home/little-prince/anaconda2/pkgs/libgcc-4.8.5-2/lib/libgomp.so.1
```
运行  
```
strings /usr/lib/x86_64-linux-gnu/libgomp.so.1 | grep GOMP_4.0
```
得到  
```
GOMP_4.0
GOMP_4.0.1
```
说明此文件就是我们想要的。接着执行
```
rm /home/little-prince/anaconda2/bin/../lib/libgomp.so.1
cp /usr/lib/x86_64-linux-gnu/libgomp.so.1 /home/little-prince/anaconda2/bin/../lib/libgomp.so.1
```


## 用xgboost训练分类模型

下段代码演示了如何用xgboost训练一个分类器并完成预测：


In [2]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import xgboost as xgb

# loading database
iris = load_iris()
# split data into train set and test set
iris_train, iris_test, y_train, y_test = train_test_split(iris['data'], iris['target'], test_size=0.33, random_state=42)
# doing all the XGBoost magic
xgb_model = xgb.XGBClassifier().fit(iris_train, y_train)
# make predicts on test set
y_pred = xgb_model.predict(iris_test)
# show accuracy
print "accuracy is:",accuracy_score(y_test, y_pred)

accuracy is: 0.98
